# Prediction for 2018-12-24 to 2018-12-31 (Testing)

In [1]:
import pandas as pd
from prophet.plot import plot_plotly, plot_components_plotly,add_changepoints_to_plot
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
df = pd.read_csv('/content/historical_weather.csv')

Checking NaN values

In [3]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182338 entries, 0 to 182337
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   city_id             182338 non-null  object 
 1   date                182338 non-null  object 
 2   avg_temp_c          181114 non-null  float64
 3   min_temp_c          176452 non-null  float64
 4   max_temp_c          174845 non-null  float64
 5   precipitation_mm    112594 non-null  float64
 6   snow_depth_mm       12238 non-null   float64
 7   avg_wind_dir_deg    146944 non-null  float64
 8   avg_wind_speed_kmh  159866 non-null  float64
dtypes: float64(7), object(2)
memory usage: 12.5+ MB


city_id                    0
date                       0
avg_temp_c              1224
min_temp_c              5886
max_temp_c              7493
precipitation_mm       69744
snow_depth_mm         170100
avg_wind_dir_deg       35394
avg_wind_speed_kmh     22472
dtype: int64

Checking Cities

In [4]:
unique_city_ids = df['city_id'].unique()
print(unique_city_ids)

['C001' 'C002' 'C003' 'C004' 'C005' 'C007' 'C008' 'C009' 'C010' 'C011'
 'C012' 'C013' 'C014' 'C015' 'C016' 'C017' 'C018' 'C020' 'C022' 'C023'
 'C024' 'C025' 'C027' 'C028' 'C029' 'C030' 'C031' 'C033' 'C034' 'C035'
 'C036' 'C037' 'C038' 'C039' 'C040' 'C042' 'C043' 'C044' 'C045' 'C046'
 'C047' 'C048' 'C049' 'C051' 'C053' 'C054' 'C055' 'C056' 'C057' 'C058'
 'C059' 'C061' 'C062' 'C064' 'C065' 'C066' 'C067' 'C068' 'C069' 'C070'
 'C071' 'C072' 'C073' 'C074' 'C076' 'C077' 'C078' 'C079' 'C081' 'C082'
 'C083' 'C084' 'C085' 'C086' 'C087' 'C088' 'C089' 'C090' 'C091' 'C092'
 'C093' 'C094' 'C095' 'C096' 'C097' 'C098' 'C099' 'C100' 'C101' 'C102'
 'C103' 'C104' 'C105' 'C106' 'C107' 'C108' 'C109' 'C110' 'C111' 'C112']


Group the DataFrame by city

In [5]:
grouped = df.groupby('city_id')

# Create a dictionary to hold DataFrames for each city
city_datasets = {city_id: group_df for city_id, group_df in grouped}
for city_id, city_df in city_datasets.items():
    print(f"City ID: {city_id}")
    print(city_df.head())

City ID: C001
  city_id        date  avg_temp_c  min_temp_c  max_temp_c  precipitation_mm  \
0    C001  2014-01-01         6.6        -1.4        11.6               NaN   
1    C001  2014-01-02         9.3         6.3        13.3               NaN   
2    C001  2014-01-03         7.6         1.9        14.0               NaN   
3    C001  2014-01-04         7.6         3.9        13.3               NaN   
4    C001  2014-01-05         8.6         0.5        16.9               NaN   

   snow_depth_mm  avg_wind_dir_deg  avg_wind_speed_kmh  
0            NaN             168.0                 6.2  
1            NaN             155.0                10.0  
2            NaN               NaN                 5.8  
3            NaN             291.0                11.3  
4            NaN               NaN                 5.0  
City ID: C002
     city_id        date  avg_temp_c  min_temp_c  max_temp_c  \
1826    C002  2014-01-01        16.6         NaN         NaN   
1827    C002  2014-01-02   

Checking City 01

In [6]:
df=city_datasets['C001'] #change this for according to city_id
df.head(30)

,city_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh
0,C001,2014-01-01,6.6,-1.4,11.6,NaN,NaN,168.0,6.2
1,C001,2014-01-02,9.3,6.3,13.3,NaN,NaN,155.0,10.0
2,C001,2014-01-03,7.6,1.9,14.0,NaN,NaN,NaN,5.8
3,C001,2014-01-04,7.6,3.9,13.3,NaN,NaN,291.0,11.3
4,C001,2014-01-05,8.6,0.5,16.9,NaN,NaN,NaN,5.0
5,C001,2014-01-06,9.1,5.7,14.7,NaN,NaN,156.0,5.7
6,C001,2014-01-07,8.8,2.0,18.8,NaN,NaN,158.0,6.2
7,C001,2014-01-08,8.8,3.0,16.3,NaN,NaN,NaN,NaN
8,C001,2014-01-09,9.9,3.9,17.1,NaN,NaN,NaN,4.8
9,C001,2014-01-10,10.8,7.0,17.2,NaN,NaN,149.0,5.8


Checking NaN values of city 01

In [7]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 1826 entries, 0 to 1825
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   city_id             1826 non-null   object 
 1   date                1826 non-null   object 
 2   avg_temp_c          1826 non-null   float64
 3   min_temp_c          1824 non-null   float64
 4   max_temp_c          1823 non-null   float64
 5   precipitation_mm    356 non-null    float64
 6   snow_depth_mm       0 non-null      float64
 7   avg_wind_dir_deg    1805 non-null   float64
 8   avg_wind_speed_kmh  1818 non-null   float64
dtypes: float64(7), object(2)
memory usage: 207.2+ KB


city_id                  0
date                     0
avg_temp_c               0
min_temp_c               2
max_temp_c               3
precipitation_mm      1470
snow_depth_mm         1826
avg_wind_dir_deg        21
avg_wind_speed_kmh       8
dtype: int64

Filling NaN values using Seasonal Decomposition

In [8]:
df = city_datasets['C001']

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Set date as index
df.set_index('date', inplace=True)

# fill NaN values using seasonal decomposition
def fill_na_with_seasonal_decomposition(series, period=7):
    series_filled = series.fillna(method='ffill').fillna(method='bfill')
    decomposition = seasonal_decompose(series_filled, model='additive', period=period)
    filled_residuals = decomposition.resid.interpolate(method='linear')
    return (decomposition.trend + decomposition.seasonal + filled_residuals).fillna(method='bfill').fillna(method='ffill')

df['avg_temp_c'] = fill_na_with_seasonal_decomposition(df['avg_temp_c'])
df['min_temp_c'] = fill_na_with_seasonal_decomposition(df['min_temp_c'])
df['max_temp_c'] = fill_na_with_seasonal_decomposition(df['max_temp_c'])
df['avg_wind_dir_deg'] = fill_na_with_seasonal_decomposition(df['avg_wind_dir_deg'])
df['avg_wind_speed_kmh'] = fill_na_with_seasonal_decomposition(df['avg_wind_speed_kmh'])
df['precipitation_mm'] = fill_na_with_seasonal_decomposition(df['precipitation_mm'])

# Special handling for snow_depth_mm if it has too many NaNs
if df['snow_depth_mm'].isna().sum() < len(df):
    df['snow_depth_mm'] = fill_na_with_seasonal_decomposition(df['snow_depth_mm'])
else:
    df['snow_depth_mm'].fillna(0, inplace=True)

# Ensure no NaN values remain
df.fillna(0, inplace=True)

# Reset the index to get back the date column
df.reset_index(inplace=True)

df.tail(30)

,date,city_id,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh
1796,2018-12-02,C001,12.2,9.0,18.0,0.0,0.0,175.0,4.9
1797,2018-12-03,C001,11.6,6.0,19.0,0.0,0.0,153.0,4.3
1798,2018-12-04,C001,11.9,8.0,14.0,10.4,0.0,155.0,10.4
1799,2018-12-05,C001,10.8,6.0,19.0,0.0,0.0,130.0,4.7
1800,2018-12-06,C001,12.1,6.0,19.0,0.9,0.0,158.0,6.5
1801,2018-12-07,C001,12.2,8.0,19.0,5.1,0.0,141.0,6.0
1802,2018-12-08,C001,13.9,11.0,19.0,10.6,0.0,146.0,8.4
1803,2018-12-09,C001,14.4,9.0,21.0,1.7,0.0,164.0,10.4
1804,2018-12-10,C001,16.1,13.0,20.0,24.0,0.0,161.0,13.7
1805,2018-12-11,C001,13.3,8.0,16.0,64.6,0.0,165.0,13.3


Checking NaN values

In [9]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                1826 non-null   datetime64[ns]
 1   city_id             1826 non-null   object        
 2   avg_temp_c          1826 non-null   float64       
 3   min_temp_c          1826 non-null   float64       
 4   max_temp_c          1826 non-null   float64       
 5   precipitation_mm    1826 non-null   float64       
 6   snow_depth_mm       1826 non-null   float64       
 7   avg_wind_dir_deg    1826 non-null   float64       
 8   avg_wind_speed_kmh  1826 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 128.5+ KB


date                  0
city_id               0
avg_temp_c            0
min_temp_c            0
max_temp_c            0
precipitation_mm      0
snow_depth_mm         0
avg_wind_dir_deg      0
avg_wind_speed_kmh    0
dtype: int64

Traing Model

In [10]:
df=df[['date','avg_temp_c','min_temp_c','max_temp_c','avg_wind_speed_kmh','avg_wind_dir_deg','precipitation_mm','snow_depth_mm']]
df.columns=['ds','y','x1','x2','x3','x4','x5','x6']
#spliting data in to train and test set
train = df.iloc[:len(df)-7]
test = df.iloc[len(df)-7:]

m = Prophet()

m.add_regressor('x1')
m.add_regressor('x2')
m.add_regressor('x3')
m.add_regressor('x4')
m.add_regressor('x5')
m.add_regressor('x6')

m.fit(train)
future = m.make_future_dataframe(periods=7) #MS for monthly, H for hourly
future['x1']=df['x1']
future['x2']=df['x2']
future['x3']=df['x3']
future['x4']=df['x4']
future['x5']=df['x5']
future['x6']=df['x6']

forecast = m.predict(future)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp62l1qoi7/gdmm50ba.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp62l1qoi7/8csp2dbc.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=59801', 'data', 'file=/tmp/tmp62l1qoi7/gdmm50ba.json', 'init=/tmp/tmp62l1qoi7/8csp2dbc.json', 'output', 'file=/tmp/tmp62l1qoi7/prophet_modelt4p5j_8x/prophet_model-20240624153844.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
15:38:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
15:38:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Predict last 7 days

In [11]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

,ds,yhat,yhat_lower,yhat_upper
1819,2018-12-25,9.603473,8.738498,10.527238
1820,2018-12-26,9.632807,8.743707,10.601887
1821,2018-12-27,9.568970,8.646046,10.395633
1822,2018-12-28,10.285057,9.442158,11.210867
1823,2018-12-29,10.294526,9.433162,11.234145
1824,2018-12-30,10.383229,9.499272,11.276064
1825,2018-12-31,10.326634,9.434278,11.182196


Actual last 7 days

In [12]:
test.tail(7)

,ds,y,x1,x2,x3,x4,x5,x6
1819,2018-12-25,9.0,3.0,18.0,4.5,151.0,0.0,0.0
1820,2018-12-26,9.4,3.0,18.0,5.9,163.0,0.0,0.0
1821,2018-12-27,9.7,4.0,17.0,6.2,157.0,0.0,0.0
1822,2018-12-28,10.1,5.7,17.0,5.2,188.0,0.0,0.0
1823,2018-12-29,10.1,5.7,17.0,5.2,188.0,0.0,0.0
1824,2018-12-30,10.1,5.7,17.0,5.2,188.0,0.0,0.0
1825,2018-12-31,10.1,5.7,17.0,5.2,188.0,0.0,0.0


In [13]:
plot_plotly(m ,forecast)

Finding RMSE

In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt


predicted = forecast['yhat'][-7:].values
actual = test['y'].values

# Calculate MAE, MSE, and RMSE
mae = mean_absolute_error(actual, predicted)
mse = mean_squared_error(actual, predicted)
rmse = sqrt(mse)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

MAE: 0.2652507094826938
MSE: 0.09131641564149039
RMSE: 0.3021860612958354


Checking Correlations

In [15]:
import pandas as pd

df = city_datasets['C001']

numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
correlations = df[numeric_columns].corr()['avg_temp_c'].sort_values(ascending=False)
print(correlations)


avg_temp_c            1.000000
max_temp_c            0.876401
min_temp_c            0.803414
avg_wind_speed_kmh    0.193855
precipitation_mm      0.061854
avg_wind_dir_deg     -0.174798
snow_depth_mm              NaN
Name: avg_temp_c, dtype: float64


# Prediction for 2018-12-31 to 2019-01-07 (Submission)

predict the next 7 days for each column

In [ ]:
import pandas as pd
from prophet import Prophet


df = pd.read_csv('/content/historical_weather.csv')

# Group the DataFrame by 'city_id'
grouped = df.groupby('city_id')
city_datasets = {city_id: group_df for city_id, group_df in grouped}

df = city_datasets['C001']

#fill NaN values using seasonal decomposition
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

def fill_na_with_seasonal_decomposition(series, period=7):
    series_filled = series.fillna(method='ffill').fillna(method='bfill')
    decomposition = seasonal_decompose(series_filled, model='additive', period=period)
    filled_residuals = decomposition.resid.interpolate(method='linear')
    return (decomposition.trend + decomposition.seasonal + filled_residuals).fillna(method='bfill').fillna(method='ffill')

df['avg_temp_c'] = fill_na_with_seasonal_decomposition(df['avg_temp_c'])
df['min_temp_c'] = fill_na_with_seasonal_decomposition(df['min_temp_c'])
df['max_temp_c'] = fill_na_with_seasonal_decomposition(df['max_temp_c'])
df['avg_wind_dir_deg'] = fill_na_with_seasonal_decomposition(df['avg_wind_dir_deg'])
df['avg_wind_speed_kmh'] = fill_na_with_seasonal_decomposition(df['avg_wind_speed_kmh'])
df['precipitation_mm'] = fill_na_with_seasonal_decomposition(df['precipitation_mm'])
if df['snow_depth_mm'].isna().sum() < len(df):
    df['snow_depth_mm'] = fill_na_with_seasonal_decomposition(df['snow_depth_mm'])
else:
    df['snow_depth_mm'].fillna(0, inplace=True)

df.fillna(0, inplace=True)
df.reset_index(inplace=True)


# Columns to forecast
columns_to_forecast = ['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm', 'snow_depth_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh']

# Initialize the new DataFrame
future_dates = pd.date_range(start='2019-01-01', periods=7, freq='D')
new_df = pd.DataFrame({'date': future_dates, 'city_id': 'C040'})

# Train and predict for each column
for column in columns_to_forecast:
    print(f"Training model for {column}")
    temp_df = df[['date', column]].rename(columns={'date': 'ds', column: 'y'})

    # Train the model
    m = Prophet()
    m.fit(temp_df)

    # Make future predictions
    future = m.make_future_dataframe(periods=7)
    forecast = m.predict(future)

    # Extract the predictions for the next 7 days
    forecast = forecast[['ds', 'yhat']].tail(7).reset_index(drop=True)
    new_df[column] = forecast['yhat']

# Concat the previous dataframe used to train the models and this new dataframe
df['city_id'] = 'C001'

# Rename 'date' to 'ds' in the original dataframe to match the new dataframe
df = df.rename(columns={'date': 'ds'})
df = df[['ds', 'city_id'] + columns_to_forecast]

# Concatenate the original and new dataframes
final_df = pd.concat([df, new_df.rename(columns={'date': 'ds'})], ignore_index=True)

# Ensure no NaN values remain
final_df.fillna(0, inplace=True)
final_df.reset_index(inplace=True)

# Display the final dataframe
final_df.tail(14)

Predict next 7 days for avg_temp_c

Predict last 7 days

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

Actual last 7 days

In [ ]:
final_df.tail(7)

In [ ]:
plot_plotly(m ,forecast)

Automate above process for each city and save the submission file

In [ ]:
import pandas as pd
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

df = pd.read_csv('/content/historical_weather.csv')

# Group the DataFrame by 'city_id'
grouped = df.groupby('city_id')

city_datasets = {city_id: group_df for city_id, group_df in grouped}

#fill NaN values using seasonal decomposition
def fill_na_with_seasonal_decomposition(series, period=7):
    series_filled = series.fillna(method='ffill').fillna(method='bfill')
    decomposition = seasonal_decompose(series_filled, model='additive', period=period)
    filled_residuals = decomposition.resid.interpolate(method='linear')
    return (decomposition.trend + decomposition.seasonal + filled_residuals).fillna(method='bfill').fillna(method='ffill')

# Prepare the submission DataFrame
submission_df = pd.DataFrame(columns=['submission_ID', 'city_id', 'date', 'avg_temp_c'])
submission_id = 1

for city_id, df in city_datasets.items():
    print(f"Processing city: {city_id}")

    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df['avg_temp_c'] = fill_na_with_seasonal_decomposition(df['avg_temp_c'])
    df['min_temp_c'] = fill_na_with_seasonal_decomposition(df['min_temp_c'])
    df['max_temp_c'] = fill_na_with_seasonal_decomposition(df['max_temp_c'])
    df['avg_wind_dir_deg'] = fill_na_with_seasonal_decomposition(df['avg_wind_dir_deg'])
    df['avg_wind_speed_kmh'] = fill_na_with_seasonal_decomposition(df['avg_wind_speed_kmh'])
    df['precipitation_mm'] = fill_na_with_seasonal_decomposition(df['precipitation_mm'])
    if df['snow_depth_mm'].isna().sum() < len(df):
        df['snow_depth_mm'] = fill_na_with_seasonal_decomposition(df['snow_depth_mm'])
    else:
        df['snow_depth_mm'].fillna(0, inplace=True)


    df.fillna(0, inplace=True)
    df.reset_index(inplace=True)

    # Columns to forecast
    columns_to_forecast = ['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm', 'snow_depth_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh']

    # Initialize the new DataFrame for predictions
    future_dates = pd.date_range(start='2019-01-01', periods=7, freq='D')
    new_df = pd.DataFrame({'date': future_dates, 'city_id': city_id})

    # Train and predict for each column
    for column in columns_to_forecast:
        print(f"Training model for {column} for city {city_id}")
        temp_df = df[['date', column]].rename(columns={'date': 'ds', column: 'y'})

        # Train the model
        m = Prophet()
        m.fit(temp_df)

        # Make future predictions
        future = m.make_future_dataframe(periods=7)
        forecast = m.predict(future)

        # Extract the predictions for the next 7 days
        forecast = forecast[['ds', 'yhat']].tail(7).reset_index(drop=True)
        new_df[column] = forecast['yhat']

    # Combine the original and new DataFrames
    df['city_id'] = city_id
    df = df.rename(columns={'date': 'ds'})

    # Reorder columns to match the new DataFrame format
    df = df[['ds', 'city_id'] + columns_to_forecast]

    # Concatenate the original and new DataFrames
    final_df = pd.concat([df, new_df.rename(columns={'date': 'ds'})], ignore_index=True)

    # Ensure no NaN values remain
    final_df.fillna(0, inplace=True)
    # Reset the index to get back the date column
    final_df.reset_index(inplace=True)

    final_df = final_df[['ds', 'avg_temp_c', 'min_temp_c', 'max_temp_c', 'avg_wind_speed_kmh', 'avg_wind_dir_deg', 'precipitation_mm', 'snow_depth_mm']]
    final_df.columns = ['ds', 'y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6']

    train = final_df.iloc[:len(final_df) - 7]
    test = final_df.iloc[len(final_df) - 7:]

    m = Prophet()

    m.add_regressor('x1')
    m.add_regressor('x2')
    m.add_regressor('x3')
    m.add_regressor('x4')
    m.add_regressor('x5')
    m.add_regressor('x6')

    m.fit(train)
    future = m.make_future_dataframe(periods=7)
    future['x1'] = final_df['x1']
    future['x2'] = final_df['x2']
    future['x3'] = final_df['x3']
    future['x4'] = final_df['x4']
    future['x5'] = final_df['x5']
    future['x6'] = final_df['x6']

    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

    # Add predicted values to submission DataFrame
    submission_data = {
        'submission_ID': range(submission_id, submission_id + 7),
        'city_id': city_id,
        'date': future_dates,
        'avg_temp_c': new_df['avg_temp_c']
    }
    submission_id += 7
    submission_df = pd.concat([submission_df, pd.DataFrame(submission_data)], ignore_index=True)

# Save to CSV
submission_df.to_csv('submission.csv', index=False)

print("Forecasting and submission file generation complete.")
